## V tem koraku bi rad preveril katera kombinacija fingerprinta in klasifikacijskega modela je najboljša preden začnem manipulirati s podatki. Ta korak je pomemben zato da v nadaljnih korakih vidim kaj izboljšujem

In [1]:
%run __A_knjiznice.py

# Import specific elements from the script
from __A_knjiznice import *
from __B_funkcije import *
%matplotlib inline

/Users/samonose/PycharmProjects/IDO_inhibitors_ML_predictions/dir1
/Users/samonose/PycharmProjects/IDO_inhibitors_ML_predictions/dir1


In [2]:

input_directory = '/Users/samonose/PycharmProjects/IDO_inhibitors_ML_predictions/dir1/molekulski_prstni_odtisi'

generated_fingerprints = ['df_circular.csv']

In [3]:
# import pandas as pd

# def remove_collinear_features_simple(x, threshold):
#     '''
#     Simplified function to remove collinear features in a DataFrame based on a specified correlation threshold.

#     Parameters:
#     - x: DataFrame with features.
#     - threshold: Correlation coefficient threshold; features with correlations above this threshold will be removed.

#     Returns:
#     - DataFrame with collinear features removed.
#     '''

#     # Calculate the correlation matrix
#     corr_matrix = x.corr().abs()

#     # Identify pairs of columns that exceed the correlation threshold
#     high_corr_pairs = [(i, j) for i in range(len(corr_matrix.columns)) for j in range(i+1, len(corr_matrix.columns)) if corr_matrix.iloc[i, j] > threshold]

#     # Create a set to hold all columns that need to be removed
#     cols_to_remove = set()

#     # Add one column from each pair to the set of columns to be removed
#     for i, j in high_corr_pairs:
#         cols_to_remove.add(corr_matrix.columns[j])

#     # Remove the columns from the DataFrame
#     x_reduced = x.drop(columns=list(cols_to_remove))

#     return x_reduced


In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold
from sklearn.cluster import FeatureAgglomeration
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA  # Import PCA
import numpy as np

# Define classifiers with default parameters
classifiers = {
    "RandomForestClassifier": RandomForestClassifier(n_jobs=-1, random_state=42)
}

# Dimensionality Reduction Methods with default parameters
dim_reduction_methods = {
    "None": None,
    "SelectKBest": SelectKBest(score_func=chi2, k=150),
    "LDA": LinearDiscriminantAnalysis(),
    "FeatureAgglomeration": FeatureAgglomeration(),
    "PCA": PCA()  # Add PCA to the methods
}

# Methods for Handling Imbalanced Data with default parameters
sampling_techniques = {
    "None": None,
    "SMOTENC": SMOTENC(categorical_features=[0, 1], random_state=42),
    "RandomUnderSampler": RandomUnderSampler(random_state=42)
}

# Store results
results_list = []

# Define Stratified k-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# List of specific filenames to process
for filename in generated_fingerprints:  # Assuming generated_fingerprints is defined
    file_path = os.path.join(input_directory, filename)
    
    if os.path.exists(file_path):  # Check if the file exists
        print(f'Processing fingerprint DataFrame: {filename}')
        
        df = pd.read_csv(file_path)
        y = df[['Activity']].values.ravel()  # Assuming 'Activity' is the target
        X = df.iloc[:, 3:]  # Assuming features start from the 4th column

        # Remove constant features
        selector = VarianceThreshold()
        X = pd.DataFrame(selector.fit_transform(X), columns=selector.get_feature_names_out())
        
        # Remove collinear features
        X = remove_collinear_features_simple(X, threshold=0.95)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, shuffle=True, stratify=y)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=15/85, random_state=42, shuffle=True, stratify=y_train)

        # Train and evaluate each classifier
        for clf_name, clf in classifiers.items():
            for dr_name, dr_method in dim_reduction_methods.items():
                for fs_name, fs_method in sampling_techniques.items():
                    steps = []
                    if fs_method is not None:
                        steps.append(('feature_selection', fs_method))
                    if dr_method is not None:
                        steps.append(('dim_reduction', dr_method))
                    steps.append(('classifier', clf))
                    
                    # Create the pipeline
                    pipeline = ImbPipeline(steps)

                    # Fit the model directly
                    pipeline.fit(X_train, y_train)

                    # Evaluate on the validation set
                    y_val_pred = pipeline.predict(X_val)
                    val_accuracy = accuracy_score(y_val, y_val_pred)
                    val_f1 = f1_score(y_val, y_val_pred)
                    val_precision = precision_score(y_val, y_val_pred)
                    val_recall = recall_score(y_val, y_val_pred)
                    val_roc_auc = roc_auc_score(y_val, y_val_pred)

                    # Append results to the list
                    results_temp = {
                        'Fingerprint': filename,  # Use the filename for identification
                        'Feature_Selection': fs_name,
                        'Dim_Reduction': dr_name,
                        'Classifier': clf_name,
                        'Val_Accuracy': val_accuracy,
                        'Val_F1': val_f1,
                        'Val_Precision': val_precision,
                        'Val_Recall': val_recall,
                        'Val_ROC_AUC': val_roc_auc,
                    }
                    results_list.append(results_temp)
                    print("\nResults:")
                    print(results_temp)

# Create DataFrame
results_df = pd.DataFrame(results_list)

Processing fingerprint DataFrame: df_circular.csv

Results:
{'Fingerprint': 'df_circular.csv', 'Feature_Selection': 'None', 'Dim_Reduction': 'None', 'Classifier': 'RandomForestClassifier', 'Val_Accuracy': 0.882636655948553, 'Val_F1': 0.8854003139717426, 'Val_Precision': 0.8785046728971962, 'Val_Recall': 0.8924050632911392, 'Val_ROC_AUC': 0.8824770414494911}

Results:
{'Fingerprint': 'df_circular.csv', 'Feature_Selection': 'SMOTENC', 'Dim_Reduction': 'None', 'Classifier': 'RandomForestClassifier', 'Val_Accuracy': 0.8810289389067524, 'Val_F1': 0.8840125391849529, 'Val_Precision': 0.8757763975155279, 'Val_Recall': 0.8924050632911392, 'Val_ROC_AUC': 0.8808430545213867}

Results:
{'Fingerprint': 'df_circular.csv', 'Feature_Selection': 'RandomUnderSampler', 'Dim_Reduction': 'None', 'Classifier': 'RandomForestClassifier', 'Val_Accuracy': 0.8762057877813505, 'Val_F1': 0.8779714738510301, 'Val_Precision': 0.8793650793650793, 'Val_Recall': 0.8765822784810127, 'Val_ROC_AUC': 0.8761996359725325}



In [5]:
results_df

,Fingerprint,Feature_Selection,Dim_Reduction,Classifier,Val_Accuracy,Val_F1,Val_Precision,Val_Recall,Val_ROC_AUC
0,df_circular.csv,None,None,RandomForestClassifier,0.882637,0.885400,0.878505,0.892405,0.882477
1,df_circular.csv,SMOTENC,None,RandomForestClassifier,0.881029,0.884013,0.875776,0.892405,0.880843
2,df_circular.csv,RandomUnderSampler,None,RandomForestClassifier,0.876206,0.877971,0.879365,0.876582,0.876200
3,df_circular.csv,None,SelectKBest,RandomForestClassifier,0.861736,0.864353,0.861635,0.867089,0.861649
4,df_circular.csv,SMOTENC,SelectKBest,RandomForestClassifier,0.861736,0.863492,0.866242,0.860759,0.861752
5,df_circular.csv,RandomUnderSampler,SelectKBest,RandomForestClassifier,0.872990,0.875197,0.873817,0.876582,0.872932
6,df_circular.csv,None,LDA,RandomForestClassifier,0.816720,0.815534,0.834437,0.797468,0.817035
7,df_circular.csv,SMOTENC,LDA,RandomForestClassifier,0.816720,0.819048,0.821656,0.816456,0.816725
8,df_circular.csv,RandomUnderSampler,LDA,RandomForestClassifier,0.831190,0.833597,0.834921,0.832278,0.831172
9,df_circular.csv,None,FeatureAgglomeration,RandomForestClassifier,0.696141,0.700475,0.701587,0.699367,0.696089


In [5]:
results_df.sort_values(by=['Val_Accuracy'], ascending=False, inplace = True)
results_df

,Fingerprint,Feature_Selection,Dim_Reduction,Classifier,Best_Params,CV_Mean_Accuracy,CV_Mean_F1,CV_Mean_Precision,CV_Mean_Recall,CV_Mean_ROC_AUC,Val_Accuracy,Val_F1,Val_Precision,Val_Recall,Val_ROC_AUC
0,df_circular.csv,None,None,RandomForestClassifier,"{'classifier__n_estimators': 100, 'classifier_...",0.880950,0.876970,0.878041,0.876199,0.934861,0.876206,0.878357,0.876972,0.879747,0.876148
2,df_circular.csv,RandomUnderSampler,None,RandomForestClassifier,"{'feature_selection__sampling_strategy': 1.0, ...",0.876464,NaN,NaN,NaN,NaN,0.874598,0.877358,0.871875,0.882911,0.874462
9,df_circular.csv,None,FeatureAgglomeration,RandomForestClassifier,"{'dim_reduction__n_clusters': 150, 'classifier...",0.876808,0.871758,0.869946,0.873815,0.930880,0.868167,0.871473,0.863354,0.879747,0.867978
1,df_circular.csv,SMOTENC,None,RandomForestClassifier,"{'feature_selection__sampling_strategy': 1.0, ...",0.876464,NaN,NaN,NaN,NaN,0.864952,0.867508,0.864780,0.870253,0.864865
3,df_circular.csv,None,SelectKBest,RandomForestClassifier,"{'dim_reduction__k': 150, 'classifier__n_estim...",0.870254,0.825374,0.839463,0.812841,0.895253,0.863344,0.865719,0.864353,0.867089,0.863283
4,df_circular.csv,SMOTENC,SelectKBest,RandomForestClassifier,"{'feature_selection__sampling_strategy': 1.0, ...",0.872323,NaN,NaN,NaN,NaN,0.860129,0.861685,0.865815,0.857595,0.860170
5,df_circular.csv,RandomUnderSampler,SelectKBest,RandomForestClassifier,"{'feature_selection__sampling_strategy': 1.0, ...",0.870252,NaN,NaN,NaN,NaN,0.860129,0.860800,0.870550,0.851266,0.860273
11,df_circular.csv,RandomUnderSampler,FeatureAgglomeration,RandomForestClassifier,"{'feature_selection__sampling_strategy': 1.0, ...",0.879225,NaN,NaN,NaN,NaN,0.860129,0.860353,0.872964,0.848101,0.860325
10,df_circular.csv,SMOTENC,FeatureAgglomeration,RandomForestClassifier,"{'feature_selection__sampling_strategy': 1.0, ...",0.872666,NaN,NaN,NaN,NaN,0.856913,0.858954,0.860317,0.857595,0.856902
6,df_circular.csv,None,LDA,RandomForestClassifier,"{'classifier__n_estimators': 50, 'classifier__...",0.825388,0.824370,0.824045,0.824937,0.868373,0.840836,0.844584,0.838006,0.851266,0.840666


## Prikažemo najboljsi model (pipeline)

In [6]:
best_model_config = results_df.iloc[0]
best_model_config

Fingerprint                                            df_circular.csv
Feature_Selection                                                 None
Dim_Reduction                                                     None
Classifier                                      RandomForestClassifier
Best_Params          {'classifier__n_estimators': 100, 'classifier_...
CV_Mean_Accuracy                                               0.88095
CV_Mean_F1                                                     0.87697
CV_Mean_Precision                                             0.878041
CV_Mean_Recall                                                0.876199
CV_Mean_ROC_AUC                                               0.934861
Val_Accuracy                                                  0.876206
Val_F1                                                        0.878357
Val_Precision                                                 0.876972
Val_Recall                                                    0.879747
Val_RO

In [7]:
# Example usage of best_model_config values
best_fingerprint = best_model_config['Fingerprint']
best_clf_name = best_model_config['Classifier']
best_dr_name = best_model_config['Dim_Reduction']
best_fs_name = best_model_config['Feature_Selection']

print(f"Combination scoring highest accuracy: \n\
Best fingerprint: {best_fingerprint}\n\
Best Classification model: {best_clf_name} \n\
Best dimensionality reduction method: {best_dr_name}\n\
Best Feature Selection model: {best_fs_name}\n\
\n\
Validation accuracy: {best_model_config['Val_Accuracy']}\n\
Training set cross validation accuracy: {best_model_config['CV_Mean_Accuracy']}")

Combination scoring highest accuracy: 
Best fingerprint: df_circular.csv
Best Classification model: RandomForestClassifier 
Best dimensionality reduction method: None
Best Feature Selection model: None

Validation accuracy: 0.8762057877813505
Training set cross validation accuracy: 0.8809499136442142


In [8]:
best_model_save= results_df.iloc[0:1]

In [9]:
dir_path = dir1/model_SU
filename = 'top_pipeline.csv'
dir_path = os.path.join(PATH, filename)
best_model_save.to_csv(dir_path,index=False)

NameError: name 'PATH' is not defined

## Reproduciramo najboljši pipeline

In [ ]:
df = df.copy()

In [ ]:
df

In [ ]:
#Best parameters found:  {'classifier__max_depth': None, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 200}

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer
from imblearn.pipeline import Pipeline as ImbPipeline
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer
from imblearn.pipeline import Pipeline as ImbPipeline
import pandas as pd
import numpy as np

# Assuming 'df' is your initial DataFrame
# Calculate fingerprints to get X and y for the best fingerprint
df_or, X = calc_fingerprints(df, best_fingerprint)
y = df_or['Activity'].values.ravel()  # Ensure y is 1D

# Remove constant features
selector = VarianceThreshold()
X = pd.DataFrame(selector.fit_transform(X), columns=selector.get_feature_names_out())

# Remove collinear features
X = remove_collinear_features_simple(X, threshold=0.95)

# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0, shuffle=True, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=15/85, random_state=0, shuffle=True, stratify=y_train)

# Rebuild the best pipeline for the selected fingerprint
steps = []
if best_fs_name != 'None':
    steps.append(('feature_selection', sampling_techniques[best_fs_name]))
if best_dr_name != 'None':
    steps.append(('dim_reduction', dim_reduction_methods[best_dr_name]))
if best_clf_name != 'None':
    steps.append(('Classifier', classifiers[best_clf_name]))

best_pipeline = ImbPipeline(steps)



# Define Stratified 5-Fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Define scorers
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'f1': make_scorer(f1_score, average='macro'),
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
}

# Calculate cross-validated metrics for the training set using parallel processing
cv_results = {}
for metric_name, scorer in scorers.items():
    cv_score = cross_val_score(best_pipeline, X_train, y_train, cv=cv, scoring=scorer, n_jobs=-1).mean()
    cv_results[f'CV_{metric_name}'] = cv_score

# Since cross_val_score fits the model internally, there's no need to fit it beforehand for CV metrics.
# Fit the model once for validation and test set predictions.
best_pipeline.fit(X_train, y_train)

# Predict on the validation set and calculate metrics
y_val_pred = best_pipeline.predict(X_val)
val_metrics = {
    'Validation_accuracy': accuracy_score(y_val, y_val_pred),
    'Validation_f1': f1_score(y_val, y_val_pred, average='macro'),
    'Validation_precision': precision_score(y_val, y_val_pred, average='macro'),
    'Validation_recall': recall_score(y_val, y_val_pred, average='macro'),
}

# Predict on the test set and calculate metrics
y_test_pred = best_pipeline.predict(X_test)
test_metrics = {
    'Test_accuracy': accuracy_score(y_test, y_test_pred),
    'Test_f1': f1_score(y_test, y_test_pred, average='macro'),
    'Test_precision': precision_score(y_test, y_test_pred, average='macro'),
    'Test_recall': recall_score(y_test, y_test_pred, average='macro'),
}

# Merge all results
all_metrics = {**cv_results, **val_metrics, **test_metrics}

# Convert to DataFrame for display
results_df = pd.DataFrame(all_metrics, index=[0])

results_df

In [ ]:
# Calculate and plot confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot using seaborn for a nicer visualization
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
if hasattr(best_pipeline, "predict_proba"):
    # Calculate probabilities for the positive class
    y_test_prob = best_pipeline.predict_proba(X_test)[:, 1]
    
    # Calculate ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test, y_test_prob)
    roc_auc = auc(fpr, tpr)
    
    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()